In [ ]:
import pandas as pd

# re-assign issue/article numbers, whoops
df = (
    pd.read_csv("./data/naut_all.csv")
    .assign(issue_number=lambda x: x.issue_title.factorize()[0] + 1)
    .assign(article_number=lambda x: x.groupby('issue_number').cumcount() + 1)
)


In [ ]:
# GCP cost?
def string_size_in_bytes(s):
    return len(s.encode('utf-8'))

# 23 million bytes
(df.article.apply(string_size_in_bytes).sum() - 1000000) * 0.000016
# ~353.800848 US for all text, less free month allowance

In [ ]:
# coqui price estimates?
from pydub import AudioSegment

import glob
from pathlib import Path

# 10 TTS
audio_files = list(Path('./data/tts_output/tacotron2_ddc_ph').rglob('*/*.mp3'))

def get_audio_length(filename):
    audio = AudioSegment.from_file(filename)
    return len(audio)  # length in milliseconds

audio_all = []
for audio_file in audio_files:
    length = get_audio_length(audio_file) / 1000.0  # convert to seconds
    audio_all.append(length)

# hours > https://coqui.ai/pricing > $20/4 hours
sum(audio_all) / 60 / 60
# 2.2028605555555556, yikes


## Tortoise Diffusion models

In [ ]:
import sys

sys.path.append('../tortoise-tts/tortoise')

import os

import torch
import torchaudio

from tortoise.api import TextToSpeech, MODELS_DIR
from tortoise.utils.audio import load_voices

voices = """angie                daniel  freeman  jlaw  myself  rainbow       tom           train_dotrice  train_grace     train_mouse
applejack            deniro  geralt   lj    pat     snakes        train_atkins  train_dreams   train_kennard   weaver
cond_latent_example  emma    halle    mol   pat2    tim_reynolds  train_daws    train_empire   train_lescault  william"""

from pathlib import Path

output_dir = Path('./data/tts_output/dev/tortoise')
output_dir.mkdir(parents=True, exist_ok=True)

voices_formatted = [word for line in voices.splitlines() for word in line.split()]
model = voices_formatted[0]

tts = TextToSpeech(models_dir=MODELS_DIR)
voice_samples, conditioning_latents = load_voices([model])

In [ ]:
from utils import nautilus_editors_note
import spacy

nlp = spacy.load('en_core_web_md')

sents = [e.text for e in list(nlp(nautilus_editors_note).sents)]

In [ ]:
for idx, sent in enumerate(sents):
    print(sent)
    gen, dbg_state = tts.tts_with_preset(sent, k=1, voice_samples=voice_samples,
                                        conditioning_latents=conditioning_latents, preset="fast",
                                        use_deterministic_seed=42, return_deterministic_state=True, cvvp_amount=.0)
    torchaudio.save(str(output_dir / f"test_{idx}.wav"), gen.squeeze(0).cpu(), 24000)

In [ ]:
# picks?
ddc

en_au_neural2_a
en_au_neural2_b

en_au_neural2_c
en_au_neural2_d

In [ ]:
model_configurations = [{'type': 'coqui',
'name': "tts_models/en/ljspeech/tacotron2-DDC",},
{'type': 'coqui',
'name': "tts_models/en/ljspeech/tacotron2-DDC",},
{'type': 'coqui',
'name': "tts_models/en/ljspeech/tacotron2-DDC",},

]

In [ ]:
from utils import to_snake_case
from pathlib import Path
from tts import tts_coqui

coqui_model = "tts_models/en/ljspeech/tacotron2-DDC_ph"
output_dir = Path("./data/tts_output/") / to_snake_case(Path(coqui_model).name)
if output_dir.exists() is False:
    output_dir.mkdir(parents=True)

for idx, row in df.head(10).iterrows():
    issue_dir = output_dir / f"{row.issue_number}_{to_snake_case(row.issue_title)}"
    if issue_dir.exists() is False:
        issue_dir.mkdir(parents=True)
    article_fp = issue_dir / f"{row.article_number}_{to_snake_case(row.headline)}.mp3"

    tts_coqui(
            coqui_model,
            row.article,
            article_fp
        )

In [ ]:
issue_dir / f"{to_snake_case(row.headline)}.mp3"